# Comparison of Statistical and Consistent Bayesian Inversion

We define a problem where the observed density corresponds to a likelihood function from classical Bayesian inversion

Copyright 2018 Michael Pilosov


In [1]:
import numpy as np
import scipy.stats as sstats
from matplotlib import pyplot as plt

In [2]:
plt.rcParams['figure.figsize'] = (20,10)
plt.rcParams['font.size'] = 18

In [3]:
import cbayes.sample as samp
import cbayes.distributions as dist
import cbayes.solve as solve
import scipy.integrate as integrate


In [4]:
import ipywidgets as wid

In [5]:
# %matplotlib inline

## Consistent Bayes

In [6]:
def comparesandbox(N = int(5E3), M = 5, pr_mean = 0, pr_sd = 0.5, ob_mean = 0.5, ob_sd = 0.5, 
                   bw = 0, p=1, cb=False, cbpf=True, sb=False, sbpf=True, pfpr=False):
    np.random.seed()
    s_set = samp.sample_set(size=(N, 1))
    s_set.set_dist('norm', kwds={'loc':pr_mean, 'scale':pr_sd}, dim=0)
#     ob_sd = np.sqrt(50./M)

    obs_data = ob_mean + ob_sd*np.random.randn(int(M))
    def QoI_fun(lam):
        try:
            q = np.power(lam.reshape(-1,1), np.float(p))
        except AttributeError:
            q = np.power(lam, np.float(p))
        if M > 1:
            residuals = q  - obs_data
            return (1./M)*np.sum( (residuals/ob_sd)**2, axis=1 ).reshape(-1,1)
        else:
            return q
    if p > 1 and bw==0:
        bw = 0.05
        
    s_set.generate_samples()
    p_set = samp.map_samples_and_create_problem(s_set, QoI_fun)
    if M == 1:
        p_set.set_observed_dist(dist='norm', dim=0, kwds={'loc':ob_mean, 'scale': ob_sd})
    else:
        p_set.set_observed_dist(dist='gamma', dim=0, kwds={'a':M/2, 'scale':2/M})
        
    if bw > 0:
        p_set.compute_pushforward_dist(method='sk', kwds={'bandwidth': bw})
    else:
        p_set.compute_pushforward_dist(method='sc') # use scipy instead if you dont care about bw (faster)
        
    # CREATE SHORT-VERSION FUNCTION HANDLES (for convenience)
    pf = p_set.pushforward_dist
    pr = p_set.prior_dist
    ob = p_set.observed_dist
    
    # Solve CBayes 
    p_set.set_ratio()
    indC = solve.perform_accept_reject(p_set.output.samples, p_set.ratio, seed=21)
    
    # Solve SBayes
    def like(lam): # Taken from Smith, pg 156
        if type(lam) is float:
            lam = np.array(lam)
        q = np.power(lam.reshape(-1, 1), np.float(p))
        residuals = q  - obs_data
        xx = (1./2)*np.sum( (residuals/ob_sd)**2, axis=1 ).reshape(-1,1)
        return np.exp(-xx)/(2*np.pi*ob_sd**2)**np.float(M/2)
    
    def S_post(x): 
        return p_set.input.dist.pdf(x)*like(x).flatten()
    
    likelihood = like(p_set.input.samples)
    evidence=integrate.quad(S_post,-3,3)
    print('Evidence: %2.4f'%evidence[0])
    indS = solve.perform_accept_reject(p_set.input.samples, likelihood, seed=225)
    print("ACCEPTED:", "SB:", len(indS), "| CB:", len(indC), " OF", N)
    
    
#     def C_post(x): # kind of works, kind of doesn't (if assumptions are violated)
#         tol = 1E-4
#         pfpr = p_set.pushforward_dist.pdf(QoI_fun(x))
#         pfpr[pfpr < tol] = 1.0
#         prr = p_set.input.dist.pdf(x)
#         obb = p_set.observed_dist.pdf(QoI_fun(x))
#         output = prr*obb/pfpr

#         output[pfpr < tol] = pfpr[pfpr < tol]
#         return output

    if len(indC) < 10:
        print(Warning("Be aware, too few accepted samples from Consistent Bayes"))
    if len(indS) < 10:
        print(Warning("Be aware, too few accepted samples from Statistical Bayes"))
    
    # SMOOTH (PUSH-FORWARDS OF) POSTERIORS FOR PLOTTING
    if len(indC)>1:
        cb_ps_den = dist.gkde(p_set.input.samples[indC])
        cb_pf_den = dist.gkde(p_set.output.samples[indC])
    else:
        print(Warning("Only one accepted sample for Consistent Bayes."))
       
    if len(indS)>1:
        sb_ps_den = dist.gkde(p_set.input.samples[indS])
        sb_pf_den = dist.gkde(p_set.output.samples[indS])
    else:
        print(Warning("Only one accepted sample for Statistical Bayes."))
    
    
    ## PLOTTING CODE
    x = np.linspace(-3,3,1000)
    LL = like(x)
    plt.plot(x, pr.pdf(x), c = 'orange', ls='--', lw=3, label='Prior')
    plt.plot(x, LL,  c='k', ls='--', lw=3, label='Statistical Likelihood Function')
    plt.scatter(np.power(ob_mean, 1./p), [0.01], facecolor='k', s=200, label='true param')
    plt.scatter(obs_data, 0*obs_data+0.01, s=50, label='data')
    
    if pfpr:
        plt.plot(x,pf.pdf(x), label='Push-forward of Prior', c='k',lw=3)

    
    if cb and len(indC)>1:
        plt.plot(x, cb_ps_den.pdf(x),  c='b', ls='-', lw=1, label='Consistent Posterior')
#         plt.plot(x, C_post(x),  c='b', ls='--', label='Analytical Consistent Posterior ')
    if cbpf and len(indC)>1:
        plt.plot(x, ob.pdf(x), label='Observed', c='r')
        plt.plot(x, cb_pf_den.pdf(x),  c='b', ls=':', lw=3, label='Consistent Posterior Push-forward')

    if sb and len(indS)>1:
        plt.plot(x, sb_ps_den.pdf(x),  c='g', ls='-', lw=1, label='Statistical Posterior')
#         plt.plot(x, S_post(x)/evidence[0],  c='g', ls='--', lw=2, label='Analytical Statistical Posterior')
    if sbpf and len(indS)>1:
        plt.plot(x, sb_pf_den.pdf(x),  c='g', ls=':', lw=3, label='Statistical Posterior Push-forward')

    
    if M == 1:
        plt.ylim([0,5])
    else:
        plt.ylim([0,5])
    plt.xlim([-2.5,2.5])
    plt.legend(loc='upper left')
    plt.title('Map with p=%d - Prior Mean at %.2f, M = %d'%(p, pr_mean, M))
#     plt.savefig('comparison.png')
    plt.show()
#     return None


In [7]:
N = wid.IntSlider(value=int(5E3), min=100, max=int(1E4), step=100, continuous_update=False)
M = wid.IntSlider(value=1, min=1, max=50, step=1, continuous_update=False)
ob_sd = wid.FloatSlider(value=0.25,  min=0.01, max=1, step=0.01, continuous_update=False)
ob_mean = wid.FloatSlider(value=1,  min=-0.5, max=1.5, step=0.25, continuous_update=False)
pr_mean = wid.FloatSlider(value=0, min=-1, max=1, step=0.05, continuous_update=False)
pr_sd = wid.FloatSlider(value=0.5,  min=0.25, max=3, step=0.05, continuous_update=False)
bw = wid.FloatSlider(value=0,  min=0, max=0.5, step=0.01, continuous_update=False)
p = wid.IntSlider(value=1, min=1, max=5, continuous_update=False)
cb = wid.Checkbox(value=True)
cbpf = wid.Checkbox(value=False)
sb = wid.Checkbox(value=True)
sbpf = wid.Checkbox(value=False)
pfpr = wid.Checkbox(value=False)

In [8]:
wid.interact_manual(comparesandbox, N=N, M=M, 
                    ob_sd=ob_sd, ob_mean=ob_mean, 
                    pr_mean=pr_mean, pr_sd=pr_sd, bw=bw, 
                    p=p, cb=cb, cbpf=cbpf, sb=sb, sbpf=sbpf, pfpr=pfpr)

interactive(children=(IntSlider(value=5000, continuous_update=False, description='N', max=10000, min=100, step…

<function __main__.comparesandbox>

## Statistical Bayes will look like the prior if the data is weak



In [9]:
M.value = 1
p.value = 1
pr_sd.value = 0.25
ob_sd.value = 2
pr_mean.value = 0
ob_mean.value = 1

## Or under a nonlinear map
We need to increase sample size to approximate the distribution well enough.
 We suggest increasing `N=10000` but even with just a few accepted samples, we recreate the observed density (which is still the same as the likelihood since `M=1` ) much more accurately.

In [10]:
N.value = 6500
M.value = 1
p.value = 5
pr_sd.value = 0.25
ob_sd.value = 0.1
pr_mean.value = 0
ob_mean.value = 0.25
bw.value = 0.05
pfpr.value = True
cb.value = False
sb.value = True
sbpf.value = False
cbpf.value = True

## With more data, the posteriors are similar (provided the signal-to-noise ratio is reasonable)
(Despite solving a different problem, the Consistent Bayes approach does appear to solve the parameter identification problem as well). 

In [11]:
N.value = 6500
M.value = 5
p.value = 5
pr_sd.value = 0.25
ob_sd.value = 0.15
pr_mean.value = 0.75
ob_mean.value = 1.0
bw.value = 0.05
pfpr.value = False
cb.value = True
sb.value = True
sbpf.value = False
cbpf.value = False

## We can separate the truth from the data and get very strange posteriors, even under the identity map.
Do note that this leads to few accepted samples.
It violates an assumption of Consistent Bayes, so the solution will make no sense. It will do its best to capture samples that are under the observed density, which occurs rarely owing to the strong bias of the prior. 
We suggest moving `ob_mean` to the left and seeing how the posteriors change.

In [12]:
M.value = 1
N.value = 5000
p.value = 1
pr_sd.value = 0.25
ob_sd.value = 0.25
pr_mean.value = 0
ob_mean.value = 1.5

## As more data comes in, Statistical Bayes starts to move towards data
Go ahead and bump `M` up even further.

In [13]:
M.value = 10

## The methods will be the same under a uniform prior.
(which we simulate by choosing a large prior standard deviation)

In [14]:
pr_sd.value = 2

## Even with less data... 

In [15]:
M.value = 1
ob_sd.value = 0.5

## Really degenerate Case - violating assumptions
Confident prior beliefs and a lot of poor quality data. More data does fix this.

In [16]:
N.value = 5E3
M.value = 10
ob_mean.value = 1
pr_mean.value = -0.5
pr_sd.value = 0.5

Confident prior beliefs and a paucity of confident data.
Very few accepted samples for cbayes, but at least it's not just basically the prior...
Note that we violate a consistent Bayesian assumption (predictability) when we do this.

In [17]:
M.value = 1
ob_mean.value = 2
pr_mean.value = 0
pr_sd.value = 0.25